In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt

import seaborn as sns 

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV


plt.rcParams["figure.figsize"] = (20, 10)

%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Загрузка данных

In [ ]:
df_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
df_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
df_train.shape

In [ ]:
df_test.shape

# Подход к решению

В первом ноутбуке мы смогли с вами быстро собрать бейзлайн, но давайте попробуем более детально посмотреть в то, какую задачу нам предстоит решать, корректно оформим подсчет метрик и поймем, как корректно проводить различные эксперименты

# Предобработка данных

Несмотря на то, что алгоритм решающего дерева умеет работать с пропущенными значениями, предлагаю более детально посмотреть на них и заполнить, так как это позволит нам глубже погрузиться в датасет и в дальнейшем сравнить работу дерева с линейной моделью:

## Обработка пропущенных значений

Посчитаем статистики по пропущенным значениям в трейне и тесте:

In [ ]:
def get_missed_values_stat(df):
    missed_stat = df.isna().sum().sort_values(ascending=False).reset_index()
    missed_stat.columns = ['feature', 'NaN count']
    missed_stat['NaN share'] = missed_stat['NaN count'] / df.shape[0]
    return missed_stat[missed_stat['NaN count'] > 0]

In [ ]:
def get_common_missed_data(df_train, df_test):
    missed_train = get_missed_values_stat(df_train)
    missed_test = get_missed_values_stat(df_test)

    missed_data = missed_train.merge(missed_test, how='outer', on='feature', suffixes=['_train', '_test'])
    
    return missed_data

Как видите, пропущенные значения есть в разных столбцах в трейне и тесте, поэтому нам будет затруднительно использовать отбрасывание столбцов с пропущенными значениями, так как могут разойтись размерности на обучении и на инференсе

Что самое важное – в тестовых данных есть пропущенные значения в тех столбцах, которые полностью заполнены в обучающей выборке, поэтому из этой ситуации надо как-то выкручиваться 

In [ ]:
missed_data = get_common_missed_data(df_train, df_test)
missed_data

### Какие признаки исключить из выборки:

Давайте исключим из выборки те признаки, доля пропущенных значений в которых больше 10%:

In [ ]:
columns_to_drop = missed_data.loc[missed_data['NaN share_train'] > 0.1, 'feature'].values
columns_to_drop

In [ ]:
df_train = df_train.drop(columns=columns_to_drop)
df_test = df_test.drop(columns=columns_to_drop)

In [ ]:
df_train.shape, df_test.shape

### Признаки `Garage*` – признаки гаража

In [ ]:
missed_data = get_common_missed_data(df_train, df_test)
missed_data

Какие признаки являются категориальными, а какие – числовыми? 

Как вы будете их заполнять?

In [ ]:
garage_cat_features = ['GarageType', 'GarageCond', 'GarageFinish', 'GarageQual']

In [ ]:
for feature in garage_cat_features:
    df_train[feature] = df_train[feature].fillna('None')
    df_test[feature] = df_test[feature].fillna('None')

In [ ]:
garage_num_features = ['GarageYrBlt', 'GarageCars', 'GarageArea']

In [ ]:
for feature in garage_num_features:
    df_train[feature] = df_train[feature].fillna(0)
    df_test[feature] = df_test[feature].fillna(0)

### Признаки `Bsmt*` – признаки подвала

In [ ]:
missed_data = get_common_missed_data(df_train, df_test)
missed_data

Какие признаки являются категориальными, а какие – числовыми? 

Как вы будете их заполнять?

In [ ]:
bsmt_cat_features = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']

In [ ]:
for feature in bsmt_cat_features:
    df_train[feature] = df_train[feature].fillna('None')
    df_test[feature] = df_test[feature].fillna('None')

In [ ]:
bsmt_num_features = ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath']

In [ ]:
for feature in bsmt_num_features:
    df_train[feature] = df_train[feature].fillna(0)
    df_test[feature] = df_test[feature].fillna(0)

### Дозаполним оставшиеся пропуски в трейне:

In [ ]:
missed_data = get_common_missed_data(df_train, df_test)
missed_data

Все признаки категориальные, кроме `MasVnrArea`:

In [ ]:
df_train[missed_data['feature']]

In [ ]:
df_train['MasVnrArea'] = df_train['MasVnrArea'].fillna(0)
df_test['MasVnrArea'] = df_test['MasVnrArea'].fillna(0)

Давайте заполним признаки не просто `None`'ами, а модами – наиболее часто встречающимися значениями, так как у нас остались единичные пропуски:

In [ ]:
df_train['Electrical'].mode()[0]

In [ ]:
df_train['Electrical'] = df_train['Electrical'].fillna(df_train['Electrical'].mode()[0])

### Дозаполним пропуски в тесте:

Это критически важно сделать, потому что иначе дерево не сможет сделать предсказание:

In [ ]:
missed_data = get_common_missed_data(df_train, df_test)
missed_data

Пропуски единичные, поэтому давайте заполним тоже наиболее часто встречающимися значениями:

In [ ]:
for feature in missed_data['feature']:
    print(feature, df_test[feature].mode()[0])
    df_test[feature] = df_test[feature].fillna(df_test[feature].mode()[0])

In [ ]:
missed_data = get_common_missed_data(df_train, df_test)
missed_data

## Работа с категориальными признаками

### Перевод числовых признаков в категориальные

In [ ]:
num_to_cat_features = ['MSSubClass', 'OverallQual', 'OverallCond']

In [ ]:
for feature in num_to_cat_features:
    df_train[feature] = df_train[feature].astype(str)
    df_test[feature] = df_test[feature].astype(str)

### Feature engineering (добавление новых признаков в данные)

Добавьте новые признаки в модель, которые смогут улучшить точность предсказания:

In [ ]:
df_train.columns

In [ ]:
def create_features(df):
    
    ## YOUR CODE HERE
    
    
    return df


# На первом запуске считаем метрики без дополнительных фичей
df_train = create_features(df_train)

### Кодирование категориальных признаков

In [ ]:
TARGET = 'SalePrice'
COLUMNS_TO_DROP = ['Id', 'SalePrice']

def preprocess_data(data, columns_to_drop, target):
    X = data.drop(columns=columns_to_drop)
    y = data[target]
    return X, y

X_train, y_train = preprocess_data(df_train, COLUMNS_TO_DROP, TARGET)
ohe = OneHotEncoder(handle_unknown='ignore')
X_train = ohe.fit_transform(X_train)

In [ ]:
X_train.shape

## Обучение модели и оценка метрик

### Подберите оптимальную глубину дерева и оцените метрики на кросс-валидации:

Подберите наилучшую глубину дерева на кросс-валидации:

In [ ]:
def log_rmse(y_true, y_pred, **kwargs):
    return mean_squared_error(np.log(y_true), np.log(y_pred), squared=False)

In [ ]:
log_rmse_scorer = make_scorer(log_rmse, greater_is_better=False)

### Сделайте подбор гиперпараметров с помощью `GridSearchCV`

#### Решающее дерево

In [ ]:
param_grid = [{'criterion': ['squared_error', 'friedman_mse'], 'max_depth': range(1, 20, 1)},
              {'criterion': ['squared_error', 'friedman_mse'], 'min_samples_leaf': range(1, 5, 1)}]

dt_reg = GridSearchCV(DecisionTreeRegressor(random_state=42), param_grid=param_grid, scoring=log_rmse_scorer)
dt_reg.fit(X_train, y_train)
print(dt_reg.best_params_)
print(dt_reg.best_score_)

## Сделайте предсказание на тестовом датасете

In [ ]:
df_test = create_features(df_test)

In [ ]:
X_test = df_test.drop(columns=COLUMNS_TO_DROP, errors='ignore')
X_test = ohe.transform(X_test)

Объект класса `GridSearchCV` делает предсказание алгоритмом с тем набором гиперпараметров, который лучше всего показал себя на кросс-валидации

Лучше себя на кросс-валидации показала линейная модель, убедитесь в том, что они будут себя также вести и на тестовой выборке, закоммитив 2 решения – с помощью дерева и с помощью линейной модели:

In [ ]:
y_test_pred = dt_reg.predict(X_test)

In [ ]:
y_test_pred 

## Закоммитьте решение в соревнование

In [ ]:
output = pd.DataFrame({'Id': df_test['Id'], 'SalePrice': y_test_pred})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")